## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,65,20,6.91,few clouds
1,1,Jacareacanga,BR,-6.2222,-57.7528,72.16,99,100,1.92,moderate rain
2,2,Cidreira,BR,-30.1811,-50.2056,70.09,85,100,7.72,overcast clouds
3,3,Jawhar,SO,2.7809,45.5005,79.29,74,12,5.39,few clouds
4,4,Ribas Do Rio Pardo,BR,-20.4431,-53.7592,72.86,67,51,4.83,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >=min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,65,20,6.91,few clouds
3,3,Jawhar,SO,2.7809,45.5005,79.29,74,12,5.39,few clouds
5,5,Kapaa,US,22.0752,-159.3190,75.20,60,40,12.66,scattered clouds
6,6,Bathsheba,BB,13.2167,-59.5167,78.80,65,40,19.57,scattered clouds
11,11,Hilo,US,19.7297,-155.0900,80.60,54,75,6.91,broken clouds
18,18,Bagaha,IN,27.1000,84.0833,77.59,35,52,8.10,broken clouds
28,28,Hithadhoo,MV,-0.6000,73.0833,81.91,63,99,4.29,overcast clouds
30,30,Rikitea,PF,-23.1203,-134.9692,77.40,68,50,5.35,scattered clouds
33,33,Mahebourg,MU,-20.4081,57.7000,75.20,94,40,6.91,light rain
40,40,Avarua,CK,-21.2078,-159.7750,84.20,65,40,8.05,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna().copy()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,65,20,6.91,few clouds
3,3,Jawhar,SO,2.7809,45.5005,79.29,74,12,5.39,few clouds
5,5,Kapaa,US,22.0752,-159.3190,75.20,60,40,12.66,scattered clouds
6,6,Bathsheba,BB,13.2167,-59.5167,78.80,65,40,19.57,scattered clouds
11,11,Hilo,US,19.7297,-155.0900,80.60,54,75,6.91,broken clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
3,Jawhar,SO,79.29,few clouds,2.7809,45.5005,
5,Kapaa,US,75.20,scattered clouds,22.0752,-159.3190,
6,Bathsheba,BB,78.80,scattered clouds,13.2167,-59.5167,
11,Hilo,US,80.60,broken clouds,19.7297,-155.0900,
18,Bagaha,IN,77.59,broken clouds,27.1000,84.0833,
28,Hithadhoo,MV,81.91,overcast clouds,-0.6000,73.0833,
30,Rikitea,PF,77.40,scattered clouds,-23.1203,-134.9692,
33,Mahebourg,MU,75.20,light rain,-20.4081,57.7000,
40,Avarua,CK,84.20,light rain,-21.2078,-159.7750,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna().copy()

City                   197
Country                197
Max Temp               197
Current Description    197
Lat                    197
Lng                    197
Hotel Name             197
dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))